### Прототип рекомендательной системы для сервиса, использующего данные Instagram

### План
- <span>&check;</span> пример сбора данных о постах
- <span>&check;</span> очистка данных
- <span>&check;</span> пример отбора тэгов
- <span>&check;</span> препроцессинг текстовых данных (текст, тэги)
- <span>&check;</span> первые рекомендации (KNN)

In [1]:
import os
import re
import json
from collections import Counter

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import NearestNeighbors

from datamining import posts_mining, likes_mining, files
from utils.preprocessing import FeatureExtractor, TextProcessing
from utils.analysis import FreqCounter, KMeansAnalyzer

In [2]:
fext = FeatureExtractor()

In [4]:
example_path = "./example_data/"

Предварительно отбираем несколько тэгов для сбора данных

В качестве примера взяты тэги для тематики мастер-классы в городе Санкт-Петербург

In [5]:
init_tags = pd.read_csv("./example_data/example_tags.csv", header=None)[0].values

In [6]:
init_tags

array(['мастерклассспб', 'мкспб'], dtype=object)

Собираем данные о постах для выбранных тэгов

In [ ]:
posts_mining.get_all_posts(init_tags, path=os.path.join(example_path, "posts", "init"), max_pages=10)

In [8]:
posts = files.merge_csv(os.path.join(example_path, "posts", "init"))

In [9]:
posts.head()

,post_id,text,date,likes,owner_id,is_video,by_tag
0,Bw1vkhAgug-,"ФРАНШИЗА ⚠️Брелок с Гос.номером⚠️\n⠀\n""Брелок ...",1556541515,26,12358401437,False,мастерклассспб
1,Bw1xA5UgTyZ,Фото GRAPHIC\nА ты хочешь без всяких заморочек...,1556542272,38,12358401437,False,мастерклассспб
2,Bw1xJKzA_8l,Коврик из гальки\nФраншиза содержит пошаговые ...,1556542340,88,12358401437,False,мастерклассспб
3,Bw6T8EJDEf0,Ни какого развода или пирамид\nНи надо зазыват...,1556694800,103,12358401437,False,мастерклассспб
4,Bw_mKSGDAYK,NaN,1556872430,95,12358401437,True,мастерклассспб


In [10]:
posts.dropna(inplace=True)

In [11]:
posts.shape

(1321, 7)

Удаляем повторяющиеся посты, используя разбиения текста на слова для большей точности

In [12]:
posts = posts.pipe(fext.drop_duplicates)

In [13]:
posts.shape

(1100, 7)

Для расширения набора данных, повторим поиск для наиболее часто встречающихся тэгов, находящихся в собранных данных, и подходящих для выбранной тематики

In [14]:
theme_pattern = re.compile("#(\w*мастеркласс\w*|мк\w*|\w*мк)(?=[#\W]|$)")
new_tags = posts["text"].map(TextProcessing(token_pat=theme_pattern).tokenize)
fc = FreqCounter().fit(new_tags)

In [15]:
fc._data.head()

,item,freq
0,мастерклассспб,413
1,мкспб,410
2,мастеркласс,199
3,мк,67
4,мкпитер,44


In [16]:
fc._data[fc._data["item"].str.contains("торт")]

,item,freq
5,мастеркласстортыпитер,31
7,мкпотортамспб,21
8,мкпотортам,21
9,мкпотортампитер,20
10,мктортыпитер,20
14,мастеркласстортыспб,12
15,мастеркласспитерторты,11
16,мастерклассспбторты,11
17,мкторты,11
23,мкпитерторты,9


Из полученных тэгов можно сделать выборку различными способами, например:
1. Использовать кластеризацию на произвольное количество групп, затем взять n случайных тэгов из каждой группы. Такой подход позволят уменьшить время на сбор данных с похожих тэгов, например "мастеркласстортыпитер" и "мкпотортамспб".
2. Из каждого кластера выбрать n кандидатов случайным образом или по наибольшим/наименьшим частотам.

In [17]:
tfidf = TfidfVectorizer(analyzer="char_wb", ngram_range=(2, 4))
tags_cluster_train = tfidf.fit_transform(fc._data["item"])
km = MiniBatchKMeans(random_state=17, n_clusters=40)
km.fit(tags_cluster_train)

random_tags = fc._data.assign(label=km.labels_).groupby("label")["item"].apply(np.random.choice)

In [18]:
print(random_tags.values)

['мастерклассповяз' 'мастерклассстеклоспб' 'мкскульптурнаяживописьспб'
 'мкброшьспб' 'мкросписьпостеклу' 'мкдетскаяшапка' 'мкпитер' 'мкшокоцветы'
 'мкпомакияжу' 'мастерклассыдлядетей' 'всемастерклассы'
 'мастерклассмакраме' 'мастеркласстортыпитер' 'мквитраж'
 'бесплатныймастеркласс' 'мксочи' 'мастерклассвязание' 'мкмосква'
 'мастерклассыгатчина' 'мастерклассповалянию' 'мкчелябинск' 'мкмозаика'
 'мкбисер' 'вышивкамк' 'мастеркласспостилю' 'мастерклассдети' 'мкоренбург'
 'эпоксиднаясмоламастеркласс' 'мкновосибирск' 'мквподарок'
 'мастерклассповязанию' 'мкмэрикэй' 'кулинарныймк' 'мкпоприческамспб'
 'мкfamilyfoto' 'самарамастеркласс' 'мккрючком' 'мкпофлористике'
 'плюшевыеигрушкимк' 'мкпокаллиграфии']


Однако в случае, когда ограничения по времени нет, можно собрать данные по всем тэгам

In [ ]:
posts_mining.get_all_posts(fc._data["item"], path=os.path.join(example_path, "posts"), max_pages=10)

In [20]:
updated_posts = files.merge_csv(os.path.join(example_path, "posts"))

In [21]:
updated_posts.dropna(inplace=True)

In [22]:
updated_posts.shape

(93837, 7)

# Фильтрация объявлений

Добавляем признаки

In [23]:
updated_posts = (updated_posts.pipe(fext.add_price)
                .pipe(fext.add_contacts)
                .pipe(fext.add_tags))

Оставляем посты, которые содержат указание хотя бы на один тип контактов (директ или телефон) или цену.

In [36]:
price_filter = updated_posts["price"] > 0
contact_filter = updated_posts[["phone_number", "direct"]].notnull().any(axis=1)

workshops = updated_posts[price_filter | contact_filter]
print(f"Number of posts with duplicates: {workshops.shape[0]}")
workshops = workshops.pipe(fext.drop_duplicates)
print(f"Number of posts without duplicates: {workshops.shape[0]}")

Number of posts with duplicates: 36055
Number of posts without duplicates: 23924


# Обработка текстовых признаков

Для предобработки текстовых данных используем класс TextProcessing в котором доступен следующий функционал:
- токенайзер с двумя режимами - обычный для слов и вероятностный (nospace) для тэгов;
- лемматизация при помощи библиотеки pymorphy2 с возможностью использовать только определенные части речи;
- удаление стоп слов.

In [37]:
%%time

non_tag_pat = "(?<![#а-я])[а-я]+(?!\S)"  # собираем данные для тэгов и текста раздельно
allowed_pos = {"ADJF", "NOUN"}  # для текста оставляем только прилагательные и существительные

text_data = TextProcessing(token_pat=non_tag_pat, allowed_pos=allowed_pos).transform(workshops["text"])

Wall time: 2min 24s


In [38]:
text_data.head()

0     [небольшой, том, сегодняшний, класс, участница...
5     [цивилизация, второе, способ, натягивание, осн...
8     [запись, живой, класс, ткачество, подробность,...
9     [небольшой, теоретический, какой, инструмент, ...
10    [класс, техника, покупка, сайт, класс, процесс...
Name: text, dtype: object

Для разделения тэгов с помощью вероятностного подхода необходимо посчитать частоты слов на максимальном количестве текстовых данных

In [39]:
%%time

non_tag_tokenizer = TextProcessing(token_pat=non_tag_pat).tokenize
counter = FreqCounter().fit(updated_posts["text"].map(non_tag_tokenizer))._freqs

Wall time: 4.85 s


Искусственно добавляем частоты для городов и других специальных слов

In [40]:
cities = pd.read_csv("./data/cities_.csv", sep=";", header=None)[1]
cities_freqs = {city: 100 for city in cities.values}

special_words = pd.read_csv("./data/special_words.csv", header=None)[1]
sp_words_freqs = {word: 100 for word in special_words}

In [53]:
counter.update(cities_freqs)
counter.update(sp_words_freqs)
print(f"Number of words in counter: {len(counter)}")

Number of words in counter: 141447


Разбиваем тэги на слова

In [43]:
%%time

tag_pat = re.compile("#([а-я]+)")

tag_data = TextProcessing(token_pat=tag_pat, mode="nospace", counter=counter).transform(workshops["tags"])

Wall time: 2min 53s


In [65]:
print(np.random.choice(tag_data))

['школа', 'флорист', 'спб', 'хороший', 'флорист', 'спб', 'флористический', 'мастерский', 'обучение', 'флористика', 'дарита', 'любимый', 'цвета', 'дарита', 'девушка', 'цвета', 'флорист', 'питер', 'метр', 'том', 'литр', 'плата', 'литр', 'том', 'плата', 'татьяна', 'москаленко', 'флорист', 'авторский', 'техника', 'курс', 'флорист', 'спб', 'хотеть', 'флорист', 'плата', 'литр', 'том', 'плата', 'обучение', 'флорист', 'спб', 'мастер', 'флорист', 'учить', 'флористика', 'преподователь', 'флористика', 'магазин', 'цветок']


Неудавшиеся разбиения убираем

In [44]:
tag_data = tag_data.map(lambda x: [w for w in x if len(w) > 2])

# Рекомендации
Делаем рекомендации по просмотру постов, используя метод ближайших соседей на векторном представлении текстовых данных

In [45]:
train_data = text_data + tag_data

In [46]:
%%time

tfidf = TfidfVectorizer(max_df=0.95, min_df=2, analyzer=lambda x: x)  # skip analysis
train_data = tfidf.fit_transform(train_data)

Wall time: 763 ms


In [47]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(train_data)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [52]:
%%time

query_index = np.random.choice(workshops.shape[0])
distances, indicies = model_knn.kneighbors(train_data[query_index], n_neighbors=6)

for i in range(len(distances.flatten())):
    if i == 0:
        print("Recs for \n{0}: \n".format(workshops["text"].iloc[query_index]))
    else:
        print("{0}: {1}, with distance of {2} \n".format(i, workshops["text"].iloc[indicies.flatten()[i]], distances.flatten()[i]))

Recs for 
Мастер-класс от Артёма Сидорова @artemsidorov
🔽
СТИЛЬ: hip-hop;
🔽
ДАТА: 26 мая 2019 / воскресенье;
🔽
ВРЕМЯ: 15:00 - 16:30;
🔽
ЦЕНА: 700 - 900 рублей;
🔽
Подробности и запись в @master.place @alexlensk;
🔽
АДРЕС: Школа танцев "Master place". Метро Садовая, переулок Гривцова, дом 6.
🔽
#всетанцыспб #мастерклассыспб #танцы #танцор #танцовщик #танцыспб #спб #питер #spb #танцывпитере #танцыпитер #танцынатнт #dance #танцуйкакчувствуешь #танцуйсвободно #безкомплексов #танцуйвсегда #танцуйвезде #люблюточтоделаю #мастеркласс #хипхоп #hiphop #мкспб26_05_19: 

1: Мастер-класс от Ankoo @ankoo_tha_ninetailz
🔽
СТИЛЬ: hip-hop;
🔽
ДАТА: 26 мая 2019 / воскресенье;
🔽
ВРЕМЯ: 19:00 - 21:00;
🔽
ЦЕНА: 600 рублей;
🔽
Подробности и запись в профиле WORKSHOPS / SOYO;
🔽
АДРЕС: Школа танцев "Master place". Метро Садовая, переулок Гривцова, дом 6.
🔽
#всетанцыспб #мастерклассыспб #танцы #танцор #танцовщик #танцыспб #спб #питер #spb #танцывпитере #танцыпитер #танцынатнт #dance #танцуйкакчувствуешь #танцуйсвободн

Источники:
<br><a href=https://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb>Токенизация тэгов с помощью вероятностного подхода</a>
<br><a href=https://link.medium.com/gvWM67L6yV>“How Did We Build Book Recommender Systems in an Hour Part 1 — The Fundamentals” by Susan Li</a>
<br><a href=https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction>Scikit-learn text feature extraction Docs</a>